# import, csv, json

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf   
## if error: python3.6 -m pip install scipy==1.2 --upgrade
import json 
import pprint
import re

In [19]:
folder = "EUW"
region = "eu"

In [20]:
df = pd.read_csv('data/{}/{}clean.csv'.format(folder,region),low_memory=False)
print(df.shape)

(14263, 1165)


# regex + participants

In [21]:
def strippart(num):
    res = []
    for name in df.columns:
        if re.search('^participants_{}_'.format(num), name) != None:
            res += [name]
    res = [re.sub('^participants_{}_'.format(num), '', x) for x in res]
    return res

In [22]:
stripped = [set(strippart(x)) for x in range(0,10)]

In [23]:
print([len(x) for x in stripped])

[106, 105, 105, 105, 105, 104, 104, 104, 104, 104]


In [24]:
common = list(set.intersection(*stripped))
print(len(common))

104


In [25]:
start = list(df.columns[:5])

team = []
for name in df.columns:
    if re.search('^team', name) != None:
        team += [name]

playercol = ['player_platformId', 'player_accountId', 'player_summonerName', 'player_summonerId', 
          'currentAccountId', 'player_matchHistoryUri', 'player_profileIcon']

partid = []
for name in df.columns:
    if re.search('^participantIdentities', name) != None:
        partid += [name]

def isolate(num):
    participant = ['participants_{}_{}'.format(num,x) for x in common]
    player = [x for x in partid if str(num) in x]
    res = df.copy()[start + team + participant + player]
    newcol = start + team + common + playercol
    res.columns = newcol
    res['participant_number'] = num
    return res

In [26]:
i = [isolate(x) for x in range(0,10)] 

In [27]:
participants = pd.concat(i)
print(participants.shape)

(142630, 161)


In [28]:
participants

,gameId,gameCreation,gameDuration,queueId,gameVersion,teams_0_win,teams_0_firstBlood,teams_0_firstTower,teams_0_firstInhibitor,teams_0_firstBaron,...,stats_perk5Var2,stats_statPerk1,player_platformId,player_accountId,player_summonerName,player_summonerId,currentAccountId,player_matchHistoryUri,player_profileIcon,participant_number
0,4681072641,1593268978835,1299,420,10.13.326.4870,0,0,0,0,0,...,0,5008,EUW1,uHf7lxqD_AHXgcQ7w8BsYZJkLQ3A32NcvNbEC9tL76_ctoU,Keny keczułoki,rkbgzOBWWOJHv8pp31OtIgvhPCsAtjAQgzyPDdUMEcYAG-M,uHf7lxqD_AHXgcQ7w8BsYZJkLQ3A32NcvNbEC9tL76_ctoU,/v1/stats/player_history/EUW1/227273950,4495,0
1,4707844099,1594758284675,1438,420,10.14.327.8505,0,0,0,0,0,...,0,5008,EUW1,R8V8agqSRpOWzXNKCQFDOe7uYYwPSRR16dTyCbHVzMRxjQ,backstreetb0ys,6lFlyTmRE9dMVT6dUaS_s_Q1I5hQ3xmkRKEtirplOfwxy0Q,R8V8agqSRpOWzXNKCQFDOe7uYYwPSRR16dTyCbHVzMRxjQ,/v1/stats/player_history/EUW1/32430502,4568,0
2,4679991311,1593201304458,1826,420,10.13.326.4870,1,1,1,1,1,...,0,5008,EUW1,7UwmsxJcxwBlpS4R5h9eN2PWuiqvi1sgKKuNTvSFXLiDuq...,losing lp,YEFgyA2pwHA-2pjUGY1OLpQkb46Fzh6szOrBbuHuyEokT5f4,7UwmsxJcxwBlpS4R5h9eN2PWuiqvi1sgKKuNTvSFXLiDuq...,/v1/stats/player_history/EUW1/2342509655527872,4571,0
3,4648730648,1591566971144,1523,420,10.11.322.2991,0,1,0,0,0,...,0,5008,EUW1,7vec5bkkSJVMm1cmPzXA-eHe21OMg78orn0ud4HeZ_jYcbY,Durum Enjoyer,O-kkiXorXHMhZD-oX9DsSkX9efVvXb_5YgZYD_ZoOuBfAIY,7vec5bkkSJVMm1cmPzXA-eHe21OMg78orn0ud4HeZ_jYcbY,/v1/stats/player_history/EUW1/224568594,22,0
4,4632641560,1590840449379,1908,420,10.11.322.2991,1,1,0,1,1,...,40,5008,EUN1,nB-Ngx_XdTR7JMljlZ3hPMcFiysKch-ldAcKQrDmEP8t7V0,Barcodee,lWz-CSIcQ6j1nuoSKbDNqmgBQK4qHJhH9ydCVg6RaPCbOIM,0KtEdshrlLyUEsIjvb5UFjJPvrKNlMcKvsVNP4qauMwdcfg,/v1/stats/player_history/EUN1/205884979,4574,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14258,4656414258,1591957338230,1417,420,10.12.324.5925,0,0,0,0,0,...,0,5008,EUW1,rvXKTSPff87zkktIOApiy4bGNuDkw-Lt82QdFlPReglFgkg,Fallen Azazel,zOznRvcKMWg5vcaEt-dXYOSakwPi1mY5LKAOOfCoBZay9X8,rvXKTSPff87zkktIOApiy4bGNuDkw-Lt82QdFlPReglFgkg,/v1/stats/player_history/EUW1/220921936,657,9
14259,4682563131,1593355545520,1479,420,10.13.326.4870,1,1,1,1,0,...,30,5008,EUW1,3yC6UhjYiy6c4hLALZZ3ZVHEndhpZaIQTU0dJVdQuVRahQ,RekSaiyajin,4JhPlXQqSQVmgev0Vk3qkKuRao2uUy-BlBcLyUsothoZ7yA,3yC6UhjYiy6c4hLALZZ3ZVHEndhpZaIQTU0dJVdQuVRahQ,/v1/stats/player_history/EUW1/29103829,4515,9
14260,4684594750,1593456408458,1970,440,10.13.326.4870,1,0,0,1,1,...,0,5008,EUW1,g8dNb1c34aPg0zXufq90xX66HdIFCIF7HAdKb9XmgSkqAG8,Shunporìna,gpcJZayACNDJK0Zp_hkxQQ1WbRpq9_Yg7QDTvvOnRAY8TCE,g8dNb1c34aPg0zXufq90xX66HdIFCIF7HAdKb9XmgSkqAG8,/v1/stats/player_history/EUW1/226356513,3785,9
14261,4675157584,1592935405722,1610,420,10.12.325.6222,1,1,1,1,1,...,0,5008,EUN1,QtDjDzYVBdup1lAaInQxvOnQZrqYBtl7K-bPivkuFQKz0YA,SPX Nafkelah,JfPnSn3rT1avRa-uA-hzevX49A5UlaDv9ITEI2JIemqQ6-6s,u7vcYcW7UTmkit3IvB1gqPmPmBjnq36AM57onjdBk_h7Tx...,/v1/stats/player_history/EUN1/208523511,4317,9


In [29]:
participants.columns

Index(['gameId', 'gameCreation', 'gameDuration', 'queueId', 'gameVersion',
       'teams_0_win', 'teams_0_firstBlood', 'teams_0_firstTower',
       'teams_0_firstInhibitor', 'teams_0_firstBaron',
       ...
       'stats_perk5Var2', 'stats_statPerk1', 'player_platformId',
       'player_accountId', 'player_summonerName', 'player_summonerId',
       'currentAccountId', 'player_matchHistoryUri', 'player_profileIcon',
       'participant_number'],
      dtype='object', length=161)

In [30]:
compression_opts = dict(method='zip',
                        archive_name='{}player.csv'.format(region))  
participants.to_csv('{}player.zip'.format(region), index=False, compression=compression_opts)

# exploration

In [23]:
# Exploring columns
for name in df.columns:
    print(name)

gameId
gameCreation
gameDuration
queueId
gameVersion
teams_0_win
teams_0_firstBlood
teams_0_firstTower
teams_0_firstInhibitor
teams_0_firstBaron
teams_0_firstDragon
teams_0_firstRiftHerald
teams_0_towerKills
teams_0_inhibitorKills
teams_0_baronKills
teams_0_dragonKills
teams_0_riftHeraldKills
teams_0_bans_0_championId
teams_0_bans_0_pickTurn
teams_0_bans_1_championId
teams_0_bans_1_pickTurn
teams_0_bans_2_championId
teams_0_bans_2_pickTurn
teams_0_bans_3_championId
teams_0_bans_3_pickTurn
teams_0_bans_4_championId
teams_0_bans_4_pickTurn
teams_1_win
teams_1_firstBlood
teams_1_firstTower
teams_1_firstInhibitor
teams_1_firstBaron
teams_1_firstDragon
teams_1_firstRiftHerald
teams_1_towerKills
teams_1_inhibitorKills
teams_1_baronKills
teams_1_dragonKills
teams_1_riftHeraldKills
teams_1_bans_0_championId
teams_1_bans_0_pickTurn
teams_1_bans_1_championId
teams_1_bans_1_pickTurn
teams_1_bans_2_championId
teams_1_bans_2_pickTurn
teams_1_bans_3_championId
teams_1_bans_3_pickTurn
teams_1_bans_4_

In [24]:
### Participants 0 - 4 are on team 0
all(df["teams_0_win"] == df["participants_4_stats_win"])

True

In [25]:
champions = df[["teams_0_win", 
                "participants_0_championId", "participants_1_championId", 
                "participants_2_championId", "participants_3_championId", 
                "participants_4_championId", "participants_5_championId", 
                "participants_6_championId", "participants_7_championId",
                "participants_8_championId", "participants_9_championId",
                "participants_0_stats_win", "participants_1_stats_win", 
                "participants_2_stats_win", "participants_3_stats_win",
                "participants_4_stats_win", "participants_5_stats_win",
                "participants_6_stats_win", "participants_7_stats_win",
                "participants_8_stats_win","participants_9_stats_win",
                "participants_0_timeline_lane", "participants_1_timeline_lane", 
                "participants_2_timeline_lane", "participants_3_timeline_lane", 
                "participants_4_timeline_lane", "participants_5_timeline_lane", 
                "participants_6_timeline_lane", "participants_7_timeline_lane",
                "participants_8_timeline_lane", "participants_9_timeline_lane",]].copy()

In [26]:
champions

,teams_0_win,participants_0_championId,participants_1_championId,participants_2_championId,participants_3_championId,participants_4_championId,participants_5_championId,participants_6_championId,participants_7_championId,participants_8_championId,...,participants_0_timeline_lane,participants_1_timeline_lane,participants_2_timeline_lane,participants_3_timeline_lane,participants_4_timeline_lane,participants_5_timeline_lane,participants_6_timeline_lane,participants_7_timeline_lane,participants_8_timeline_lane,participants_9_timeline_lane
0,1,Leblanc,Hecarim,Ashe,Thresh,Rengar,Sett,Graves,Aatrox,Syndra,...,MIDDLE,JUNGLE,BOTTOM,BOTTOM,JUNGLE,MIDDLE,JUNGLE,TOP,MIDDLE,JUNGLE
1,1,Volibear,Caitlyn,Taliyah,Jayce,Leblanc,Swain,LeeSin,Zoe,Sett,...,JUNGLE,BOTTOM,BOTTOM,TOP,MIDDLE,BOTTOM,JUNGLE,MIDDLE,BOTTOM,TOP
2,1,Chogath,Ekko,RekSai,Aphelios,Sett,LeeSin,Taliyah,Alistar,Syndra,...,TOP,TOP,JUNGLE,BOTTOM,MIDDLE,JUNGLE,BOTTOM,BOTTOM,MIDDLE,TOP
3,0,Lux,Caitlyn,Elise,Galio,Qiyana,Graves,Pyke,Orianna,Camille,...,BOTTOM,BOTTOM,JUNGLE,TOP,MIDDLE,JUNGLE,BOTTOM,BOTTOM,TOP,MIDDLE
4,0,Xayah,Jayce,Sylas,Senna,Trundle,Maokai,Nocturne,Kaisa,Azir,...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27475,1,Sett,Kaisa,Jax,LeeSin,TwistedFate,Kassadin,Nidalee,Caitlyn,Camille,...,BOTTOM,BOTTOM,TOP,JUNGLE,MIDDLE,MIDDLE,JUNGLE,BOTTOM,MIDDLE,BOTTOM
27476,0,Olaf,Ashe,Sett,Zoe,Graves,Talon,Karthus,Volibear,Rakan,...,TOP,BOTTOM,BOTTOM,MIDDLE,JUNGLE,MIDDLE,JUNGLE,MIDDLE,BOTTOM,BOTTOM
27477,1,Lucian,Zac,Sylas,Soraka,Ezreal,Senna,Volibear,LeeSin,Aphelios,...,MIDDLE,JUNGLE,TOP,MIDDLE,BOTTOM,BOTTOM,TOP,JUNGLE,BOTTOM,MIDDLE
27478,1,Camille,Qiyana,Leona,Vayne,Karthus,Sett,Kaisa,Yasuo,Volibear,...,TOP,MIDDLE,BOTTOM,BOTTOM,JUNGLE,BOTTOM,BOTTOM,MIDDLE,TOP,JUNGLE


In [27]:
#series of total games per champ
champgames = []
for col in champions.columns:
    champgames.append(champions[col].value_counts())

totalgames = champgames[1]
for i in range(2,10):
    totalgames += champgames[i]

In [28]:
#series of win percentages per champ
champwins = {}
for i in range(0,5):
    champwins[i] = champions.groupby('participants_{}_championId'.format(i)
                                    ).mean()['teams_0_win']
for i in range(5,10):
    champwins[i] = 1 - champions.groupby('participants_{}_championId'.format(i)
                                        ).mean()['teams_0_win']    
totalwins = pd.DataFrame(champwins).mean(axis=1)

In [29]:
#create df
wins = totalwins.to_frame().join(totalgames.to_frame())
wins.columns = ['win_percent','num_games']
wins

,win_percent,num_games
876,0.470992,947
Aatrox,0.498911,1945
Ahri,0.540395,582
Akali,0.471929,2081
Alistar,0.511899,1357
...,...,...
Zed,0.485721,1662
Ziggs,0.475232,222
Zilean,0.487093,517
Zoe,0.504925,3468


In [38]:
import requests
import re

In [48]:
summoner = requests.get("http://ddragon.leagueoflegends.com/cdn/10.15.1/data/en_US/summoner.json").json()

In [49]:
items = requests.get("http://ddragon.leagueoflegends.com/cdn/10.15.1/data/en_US/item.json").json()

In [58]:
items['data']['1011']

{'name': "Giant's Belt",
 'description': '<stats>+380 Health</stats>',
 'colloq': ';',
 'plaintext': 'Greatly increases Health',
 'from': ['1028'],
 'into': ['3083', '3084', '3022', '3143', '3742'],
 'image': {'full': '1011.png',
  'sprite': 'item0.png',
  'group': 'item',
  'x': 144,
  'y': 0,
  'w': 48,
  'h': 48},
 'gold': {'base': 600, 'purchasable': True, 'total': 1000, 'sell': 700},
 'tags': ['Health'],
 'maps': {'11': True, '12': True, '21': True, '22': False},
 'stats': {'FlatHPPoolMod': 380},
 'depth': 2}

In [45]:
participants[[col for col in participants.columns if 'perk' in col]]

,stats_perk1Var1,stats_perk4Var3,stats_perk2,stats_perk0Var2,stats_perk0Var1,stats_perk3Var2,stats_perk1Var2,stats_perk2Var1,stats_perk4Var1,stats_perk4Var2,...,stats_perk5Var2,stats_perk3Var1,stats_perk3Var3,stats_perk2Var2,stats_perk4,stats_perk0Var3,stats_perk2Var3,stats_perkPrimaryStyle,stats_perk5Var1,stats_perkSubStyle
0,1011,0,8138,0,1848,5,0,30,1258,480,...,0,4590,0,0,9111,0,0,8100,921,8000
1,674,0,9105,616,1088,0,120,17,502,0,...,0,557,0,30,8473,471,0,8000,0,8400
2,1089,0,9105,0,817,0,300,15,5736,5,...,0,850,0,30,8135,0,0,8000,510,8100
3,793,0,8138,0,1033,4,0,18,250,828,...,0,2168,0,0,8226,0,0,8100,0,8200
4,250,0,8210,0,17,0,1060,16,756,0,...,4,382,0,0,8139,0,0,8200,4425,8100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29468,0,0,9104,0,145,0,0,0,3,0,...,1410,66,0,0,8345,0,0,8000,230,8300
29469,0,0,8444,0,84,0,0,147,0,0,...,0,0,0,0,8313,0,0,8400,0,8300
29470,250,0,8233,1721,720,0,321,6,436,0,...,478,24,0,50,8463,0,0,8200,697,8400
29471,250,0,8210,0,2256,0,486,0,3,0,...,0,736,0,0,8345,0,0,8200,0,8300


# fit

In [ ]:
#results = smf.logit('teams_0_win ~ participantIdentities_1_player_accountId', data=df).fit()
#results.summary()